In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [ ]:
import findspark

# them file jar
# findspark.add_packages("org.elasticsearch:elasticsearch-spark-30_2.12:7.12.0,org.apache.httpcomponents:httpclient:4.1.1")
findspark.init('/opt/spark320') # để ăn config spark 

conf = SparkConf().setAppName('check hadoop')
conf.set('spark.yarn.queue', 'plat')
conf.set('spark.kerberos.principal', 'long/plat@HADOOP.SECURE')
conf.set('spark.kerberos.keytab', 'path')
conf.set('spark.kerberos.access.hadoopFileSystems', 'hdfs://172.1.1.1:9000')
conf.set('spark.sql.parquet.binaryAsString', 'true') # nếu không set thì có thể có lúc đọc file ra binary type

conf.set('spark.port', '4071')

conf.set('spark.driver.memory', '2g') 
conf.set('spark.executor.memory', '2G')

spark = SparkSession.builder.master("local[8]").config(conf=conf).getOrCreate()

# conf.set('spark.cores.max', '1')
# conf.set('spark.executor.instances', '5')
# spark = SparkSession.builder.master("yarn").config(conf=conf).getOrCreate()


spark.sparkContext.setLogLevel("ERROR")

In [ ]:
import os
import sys

# $ conda activate env_36 
# $ source /opt/spark320/conf/spark-env.sh # chú ý phải chạy lệnh trên khi tạo mới môi trường, nếu k spark trên yarn sẽ k ăn config PYSPARK_PYTHON và PYSPARK_DRIVER_PYTHON của spark
# $ jupyter-notebook --ip=0.0.0.0 --port=9090

print(os.environ['PYSPARK_PYTHON'])  # giống trong file spark-env. vì trên các executor mới có cái path này
print(os.environ['PYSPARK_DRIVER_PYTHON'])

# để config như trên thì chạy được pyspark trên yarn, tuy nhiên k  chạy được local[] vì nó đang ăn env của spark-env, k ăn path của env chạy notebook hiện tại. lệnh set lại để chạy local[]
os.environ['PYSPARK_PYTHON'] = "/storage/long/anaconda3/envs/env_36/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/storage/long/anaconda3/envs/env_36/bin/python3"

In [ ]:
df_uv = spark.read.parquet("hdfs://172.1.1.1.:9000/data/{2023_{10,11,12}_*,2024_{01,02}_*}/*")
df_uv.show()

In [ ]:
# partition

from aerospike import client # chịu k import dc
def process_partition(iter_row):

    config = {
        'hosts': [('1.1.1.1',3000)] 
    }
    client_instance = client(config).connect()
    namespace = 'mem_hadoop'
    dataset = 'user_profile'

    def get(key):
        key = (namespace, dataset, key)
        try:
            (key, metadata, record) = client_instance.get(key)
            return 1
        except Exception as e: 
    #         print("read error ", e)
            return -1

    out = []
    # Iterate over the records in the partition
    for row in iter_row: 

        user_id = row[0]
        exits = get(user_id)
        row_out = (user_id,exits)
        out.append(row_out)
        

    client_instance.close()
    
    return out


# df_uv schema : user_id, uv ...
result_rdd = df_uv.rdd.mapPartitions(process_partition)
result_rdd.cache().count()
res_df = result_rdd.toDF(("userId", "exits"))
res.filter("exits ==1").count()